# Initial

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from math import floor
from sklearn.metrics import confusion_matrix
from skmultiflow.lazy import SAMKNNClassifier
from skmultiflow.meta import AdaptiveRandomForestClassifier, BatchIncrementalClassifier, ProbabilisticClassifierChain, LearnPPNSEClassifier, LearnPPClassifier, LeveragingBaggingClassifier, MultiOutputLearner, OnlineAdaC2Classifier, OnlineBoostingClassifier, OnlineCSB2Classifier, OnlineRUSBoostClassifier, OnlineSMOTEBaggingClassifier, OnlineUnderOverBaggingClassifier, OzaBaggingClassifier, OzaBaggingADWINClassifier

In [3]:
'''
EXTERNAL KERNEL
'''
google_colab = False
kaggle = False

'''
CUDA
'''
cuda = False

'''
DATA REPRESENTATION

1 => SINGLE READ | 2 => ADD FEATURES | 3 => WINDOW TO FEATURES
'''
DATA_REPRESENTATION = 3

'''
DOWNSAMPLE FACTOR

1 => 10hz *original rate* | 2 => 5Hz | 5 => 2Hz | 10 => 1hz
'''
DOWNSAMPLE_FACTOR = 5

'''
WINDOWS LENGHT

* needs divisor by datapoints target
* considering downsample factor = 5

1 => WINDOW DISABLED | 2 => 1 second | 4 => 2 seconds | 10 => 5 seconds | 20 => 10 seconds | 200 => 100 seconds *full flight*
'''
WINDOW_LENGHT =  10

'''
LIMITADOR

Quantity of samples in the execution of the tests.
'''
LIMITADOR = None

'''
LOSS FACTOR [0,1]

Ignores outliers in calculating the stats of losses in regenerated data.
'''
LOSS_FACTOR = 1

'''
TRAIN_SIZE [0,1]

Percentage of samples to be trained
'''
TRAIN_SIZE = 0.8

'''
OUTPUT_FILE_NAME

File with output results
'''
OUTPUT_FILE_NAME = 'output_knn_skflow_dr_' + str(DATA_REPRESENTATION) + '-ts_' + str(TRAIN_SIZE) + '-lf_' + str.replace(str(LOSS_FACTOR), '.', '') + '-limit_' + str(LIMITADOR) + '-wl_' + str(WINDOW_LENGHT) + '.txt'

'''
PATH_OUTPUTS

local : ./outputs/
google colab : /content/drive/My Drive/
'''
PATH_OUTPUTS = '/content/drive/My Drive/'


'''
PATH_DATASET

'''
PATH_DATASET = '../../dataset/original/'

'''
FLUSH FILE

If output results file is ON
'''
FLUSH_FILE = False

In [4]:
if google_colab:
    !pip install git+https://github.com/online-ml/river --upgrade

    from google.colab import drive

    drive.mount('/content/drive')
    path = '/content/drive/My Drive/ACADÊMICO/MESTRADO/DISSERTAÇÃO/CHAPTERS/5 EXPERIMENTO/dataset/data_representation_1'
    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv(path+'/F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv(path+'/F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv(path+'/F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv(path+'/F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv(path+'/F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv(path+'/F16_DS3_fault4_fixedposition.csv', header=None),
    }
elif kaggle:
    !conda install -y gdown
    !gdown --id 1G88okIVmdcgLFlmd7rDRhHvHv98yK3UB
    !gdown --id 1fX3utfHMjwKTt7IW4D01bnm-hv88yzrJ
    !gdown --id 1yUG3R5zK2AIxtS9Q4Fk-udkKBZeYShgb
    !gdown --id 1OBRDtuqNEZ-3Z-q0helWh2xGiAxeLACH
    !gdown --id 17oDi60sWYsWHHxzj2aA9m6ARm8zQ81m_
    !gdown --id 1jKEK4s5sYJh8PHtpHeV8ABOsHjuB26RA
else:
    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault4_fixedposition.csv', header=None),
    }

# Dataset

In [5]:
dict_ds = dict_ds_original.copy()

if dict_ds['data_ds3_normal_t1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0 or dict_ds['data_ds3_fault1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0:
    raise Exception('Needs to be ?shape? divisor')

for n, dataset_name in enumerate(dict_ds):
    dataset = dict_ds[dataset_name].to_numpy()

    downsampled = dataset[::DOWNSAMPLE_FACTOR]

    x, y = downsampled.shape

    # resample
    dict_ds[dataset_name] = pd.DataFrame(downsampled.reshape((int(x/WINDOW_LENGHT),y*WINDOW_LENGHT)))


In [6]:
# ADD COLUMNS WITH DIFF PREVIOUS VALUES

if (DATA_REPRESENTATION == 2):
    frame_size = int(1000/DOWNSAMPLE_FACTOR)

    for n, dataset_name in enumerate(dict_ds):
        dataset = dict_ds[dataset_name].to_numpy()

        dimension = dataset.shape[1]
        samples = dataset.shape[0]

        # GENERATE NEW DIMENSIONS
        dataset = np.concatenate((dataset, np.zeros((samples,dimension))), axis=1)

        for f in np.arange(0,int(samples/frame_size)):
            # OBTAIN THE FRAME FLIGHT
            frame = dataset[f*frame_size:(f+1)*frame_size, 0:dimension]

            # CALCULATE DIFFERENCE
            chunk = np.diff(frame, axis=0)

            # DONT CALCULATE THE DIFFERENCE FOR EACH FIRST TIMESTEP
            chunk = np.insert(chunk, 0, frame[0, 0:dimension], axis=0)

            # UPDATE DATASET WITH NEW FRAME INTO NEW DIMENSIONS
            dataset[f*frame_size:(f+1)*frame_size,dimension:dimension*2] = chunk

        dict_ds[dataset_name] = pd.DataFrame(dataset)



In [7]:
ss = StandardScaler()
#ss = MinMaxScaler()

data_ds3_t1_normal = dict_ds['data_ds3_normal_t1_original'].copy()
data_ds3_t2_normal = dict_ds['data_ds3_normal_t2_original'].copy()
data_ds3_fault1 = dict_ds['data_ds3_fault1_original'].copy()
data_ds3_fault2 = dict_ds['data_ds3_fault2_original'].copy()
data_ds3_fault3 = dict_ds['data_ds3_fault3_original'].copy()
data_ds3_fault4 = dict_ds['data_ds3_fault4_original'].copy()

# fit values
ss.partial_fit(data_ds3_t1_normal)
ss.partial_fit(data_ds3_t2_normal)
ss.partial_fit(data_ds3_fault1)
ss.partial_fit(data_ds3_fault2)
ss.partial_fit(data_ds3_fault3)
ss.partial_fit(data_ds3_fault4)

# transform values
data_ds3_t1_normal = ss.transform(data_ds3_t1_normal)
data_ds3_t2_normal = ss.transform(data_ds3_t2_normal)
data_ds3_fault1 = ss.transform(data_ds3_fault1)
data_ds3_fault2 = ss.transform(data_ds3_fault2)
data_ds3_fault3 = ss.transform(data_ds3_fault3)
data_ds3_fault4 = ss.transform(data_ds3_fault4)

# append normal labels
data_ds3_t1_normal = np.append(data_ds3_t1_normal, np.zeros((data_ds3_t1_normal.shape[0],1)), axis = 1)
data_ds3_t2_normal = np.append(data_ds3_t2_normal, np.zeros((data_ds3_t2_normal.shape[0],1)), axis = 1)

# append fault labels
def generate_fault_label(dataset, fault_label):
    labels = np.array([[fault_label]]*dataset.shape[0])

    return labels

data_ds3_fault1 = np.append(data_ds3_fault1, generate_fault_label(data_ds3_fault1, 1), axis = 1)
data_ds3_fault2 = np.append(data_ds3_fault2, generate_fault_label(data_ds3_fault2, 2), axis = 1)
data_ds3_fault3 = np.append(data_ds3_fault3, generate_fault_label(data_ds3_fault3, 3), axis = 1)
data_ds3_fault4 = np.append(data_ds3_fault4, generate_fault_label(data_ds3_fault4, 4), axis = 1)

# Split

In [8]:
# STRATIFIED

'''X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(data_ds3_fault1[:LIMITADOR, :-1], data_ds3_fault1[:LIMITADOR, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(data_ds3_fault2[:LIMITADOR, :-1], data_ds3_fault2[:LIMITADOR, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(data_ds3_fault3[:LIMITADOR, :-1], data_ds3_fault3[:LIMITADOR, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(data_ds3_fault4[:LIMITADOR, :-1], data_ds3_fault4[:LIMITADOR, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)


X_train = np.concatenate((X_train_1, X_train_2, X_train_3, X_train_4))
X_test = np.concatenate((X_test_1, X_test_2, X_test_3, X_test_4))
y_train = np.concatenate((y_train_1, y_train_2, y_train_3, y_train_4))
y_test = np.concatenate((y_test_1, y_test_2, y_test_3, y_test_4))'''

faults = np.concatenate((data_ds3_fault1, data_ds3_fault2, data_ds3_fault3))
X_train, X_test, y_train, y_test = train_test_split(faults, faults, test_size=1-TRAIN_SIZE, random_state=42, shuffle=True)


In [9]:
def predict(model, data):
    probas = []

    for k, x in enumerate(data):
        y_pred_proba = model.predict_proba(x.reshape(1, x.shape[0]))
        probas.append(y_pred_proba)

    return probas

def generate_y_hat(probas, threshold):
    y_hat = []

    for p in probas:
        if (np.max(p) < threshold): #indecisao
            y_hat.append(-1)
        else: #certeza
            y_hat.append(np.argmax(p))

    return y_hat

knn_p = {
    'n_neighbors': [5], # def 5
    'leaf_size': [30], # def 30
    'window_size': [1000], # def 1000
    'p': [2] # 1 Minkowski | def 2 euclidean
}

phis = [.5]


# Run

In [ ]:
'''
Increment Parameters
'''
increments = 10
block_size = int(floor(len(X_train) / increments))

log = None
if FLUSH_FILE:
    log = open(PATH_OUTPUTS+OUTPUT_FILE_NAME, "a", buffering=1)

for n in knn_p['n_neighbors']:
    for ls in knn_p['leaf_size']:
        for ws in knn_p['window_size']:
            for p in knn_p['p']:
                print('.', end='')
                '''classifier = ensemble.BaggingClassifier(
                    model=(
                            preprocessing.MinMaxScaler() |
                            neighbors.KNNADWINClassifier()
                    )
                )'''
                clf = SAMKNNClassifier(max_window_size=25)

                #for c, x in enumerate(X_train, 0): # AQUI ESTAH TODO O TREINAMENTO
                    #clf.partial_fit(x.reshape(1, x.shape[0]), [y_train[c]])

                clf.partial_fit(X_train[:50], y_train[:50])

                cm = confusion_matrix(y_test[:50], clf.predict(X_test[:50]), normalize='true')
                print(cm)

                '''print('-', end='')
                probas = predict(clf, X_test[:500])

                for phi in phis:
                    t = phi

                    y_hat = generate_y_hat(probas, t)
                    y_test[0] = -1 #forçando a existencia de -1
                    cm = confusion_matrix(y_test[:500], np.array(y_hat), normalize='true')
                    print(cm)
                    print(clf, ls, ws, p, '%.5f'%(t), file=log)
                    print('X_TEST', file=log)
                    print(0, file=log)
                    print('NORMAL_2', file=log)
                    print(0, file=log)

                    bingo = True
                    for i in np.arange(1,5):
                        print('F', i, file=log)
                        percent = cm[i,i]*100
                        print(percent, file=log)

                        if percent < 50:
                            bingo = False

                    if bingo:
                        print('BINGO', file=log)
                    else:
                        print('WATER', file=log)'''

if FLUSH_FILE:
    log.close()

.

# New Concept Run

In [12]:
faults = data_ds3_fault4
X_train2, X_test2, y_train2, y_test2 = train_test_split(faults, faults, test_size=1-TRAIN_SIZE, random_state=42, shuffle=True)


In [13]:
'''
Increment Parameters
'''
increments = 10
block_size = int(floor(len(X_train) / increments))

log = None
if FLUSH_FILE:
    log = open(PATH_OUTPUTS+OUTPUT_FILE_NAME, "a", buffering=1)

for n in knn_p['n_neighbors']:
    for ls in knn_p['leaf_size']:
        for ws in knn_p['window_size']:
            for p in knn_p['p']:
                print('.', end='')
                '''classifier = ensemble.BaggingClassifier(
                    model=(
                            preprocessing.MinMaxScaler() |
                            neighbors.KNNADWINClassifier()
                    )
                )'''


                #for c, x in enumerate(X_train, 0): # AQUI ESTAH TODO O TREINAMENTO
                #clf.partial_fit(x.reshape(1, x.shape[0]), [y_train[c]])

                clf.partial_fit(X_train2[:5000], y_train2[:5000])



                print('-', end='')
                probas = predict(clf, np.concatenate((X_test[:1500], X_test2[:500])))

                for phi in phis:
                    t = phi

                    y_hat = generate_y_hat(probas, t)
                    y_test[0] = -1 #forçando a existencia de -1
                    cm = confusion_matrix(np.concatenate((y_test[:1500],y_test2[:500])), np.array(y_hat), normalize='true')
                    print(cm)

if FLUSH_FILE:
    log.close()

.-[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [14]:
KNNClassifier.valid_metrics()

['euclidean',
 'l2',
 'minkowski',
 'p',
 'manhattan',
 'cityblock',
 'l1',
 'chebyshev',
 'infinity']